# Assignment 1 Task 2

In [44]:
import re
import string
import scipy
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import *
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from astropy.table import Table, Column
import os
import glob
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from collections import defaultdict
from sklearn.feature_selection import VarianceThreshold

# Assignment 1 Task 1

In [21]:
df = pd.DataFrame(columns=['tweet','gender'])
for f in glob.glob('Assignment_4/twitter-gender-corpus/male/*.txt'):
    fil = open(f,encoding="utf8") 
    df = df.append(pd.DataFrame({'tweet': fil.read(), 'gender':'Male'}, index=[0]) , ignore_index=True)
    
for f in glob.glob('Assignment_4/twitter-gender-corpus/female/*.txt'):
    fil = open(f,encoding="utf8") 
    df = df.append(pd.DataFrame({'tweet': fil.read(), 'gender':'Feale'}, index=[0]) , ignore_index=True)

df

,tweet,gender
0,"@zulahni Oh, honey. Don&#39;t force us to stag...",Male
1,@lukeniuk Tack män! Kul som fan att träffas!\n...,Male
2,"Report from SIGIR 2011 (Beijing, China) - http...",Male
3,After quite some time working on different thi...,Male
4,Financial crisis hurting new job seekers in Ja...,Male
...,...,...
420,@pablomoreiras Exacto ;-) @eEAmerica\nLo q me ...,Feale
421,"&quot;Chance is like a picture, it&#39;d be ni...",Feale
422,How to Recognize a Hipster - Your server has a...,Feale
423,@rigbysdeli thanks so much! Info sent\nA moto ...,Feale


# Label Encoding

In [22]:
ge = LabelEncoder()

df['gender'] = ge.fit_transform(df['gender'])
df

,tweet,gender
0,"@zulahni Oh, honey. Don&#39;t force us to stag...",1
1,@lukeniuk Tack män! Kul som fan att träffas!\n...,1
2,"Report from SIGIR 2011 (Beijing, China) - http...",1
3,After quite some time working on different thi...,1
4,Financial crisis hurting new job seekers in Ja...,1
...,...,...
420,@pablomoreiras Exacto ;-) @eEAmerica\nLo q me ...,0
421,"&quot;Chance is like a picture, it&#39;d be ni...",0
422,How to Recognize a Hipster - Your server has a...,0
423,@rigbysdeli thanks so much! Info sent\nA moto ...,0


In [25]:
df.loc[0].gender

1

#  Feature Extraction 

# Stylometry Based methods 



In [26]:
f_array = []
for idx,combine_data in enumerate(df.tweet):
    commas_counter = combine_data.count(',')
    fullstop_counter = combine_data.count('.')
    At_counter = combine_data.count('@')
    leftbracket_counter = combine_data.count('(')
    rightbracket_counter = combine_data.count(')')
    exclamation_counter = combine_data.count('!')
    minus_counter = combine_data.count('-')
    qm_counter = combine_data.count('?')
    per_counter = combine_data.count('%')
    and_counter = combine_data.count('&')
    hash_counter = combine_data.count('#')
    US_counter = combine_data.count('_')
    equal_counter = combine_data.count('=')
    capital_letters_counter=sum(1 for c in combine_data if c.isupper())
    sc_counter = combine_data.count(';')
    colons_counter = combine_data.count(':')
    space_counter = combine_data.count(' ')
    small_letters_counter=sum(1 for c in combine_data if c.islower())
    fs_counter = combine_data.count('/')
    digit_counter=sum(1 for c in combine_data if c.isdigit())
    f_array.append([commas_counter,fullstop_counter,At_counter,leftbracket_counter,rightbracket_counter,
          exclamation_counter,minus_counter,qm_counter,per_counter,and_counter,hash_counter,
          US_counter,equal_counter,capital_letters_counter,sc_counter,colons_counter,space_counter,
           small_letters_counter,fs_counter,digit_counter,df.loc[idx].gender])

    
df = pd.DataFrame(f_array, columns=['commas_counter','fullstop_counter','At_counter','leftbracket_counter',
                                    'rightbracket_counter','exclamation_counter','minus_counter','qm_counter',
                                    'per_counter','and_counter','hash_counter','US_counter','equal_counter',
                                    'capital_letters_counter','sc_counter','colons_counter','space_counter',
                                    'small_letters_counter','fs_counter','digit_counter','gender'])
df

,commas_counter,fullstop_counter,At_counter,leftbracket_counter,rightbracket_counter,exclamation_counter,minus_counter,qm_counter,per_counter,and_counter,...,US_counter,equal_counter,capital_letters_counter,sc_counter,colons_counter,space_counter,small_letters_counter,fs_counter,digit_counter,gender
0,515,1590,805,143,284,331,294,205,3,1168,...,46,23,7525,1214,686,12074,63999,1180,2322,1
1,605,872,1256,27,175,1184,265,442,8,412,...,173,9,4928,445,272,13123,64726,307,1077,1
2,339,1429,94,191,200,46,961,106,11,922,...,22,46,7319,941,1191,12478,79767,2672,2804,1
3,187,512,195,88,130,89,345,70,6,292,...,31,6,2504,327,348,9001,47792,278,1795,1
4,125,1495,66,15,17,81,858,125,15,379,...,95,89,6254,380,851,8333,56654,2499,3598,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,56,324,322,8,230,80,256,12,4,112,...,42,14,1150,308,82,2034,11628,178,350,0
421,283,1355,750,92,139,328,290,132,1,963,...,189,44,5702,963,522,10843,57688,1524,2127,0
422,208,804,46,66,62,222,280,88,6,466,...,78,150,3250,466,406,4438,28746,996,1236,0
423,203,1287,852,83,94,251,745,162,51,846,...,263,89,5198,854,512,13769,80677,1625,2535,0


#  Split train, test data 

In [35]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,:20],df.iloc[:,20],test_size=0.30,random_state=50)

##  Train Data Input  

In [36]:
x_train

,commas_counter,fullstop_counter,At_counter,leftbracket_counter,rightbracket_counter,exclamation_counter,minus_counter,qm_counter,per_counter,and_counter,hash_counter,US_counter,equal_counter,capital_letters_counter,sc_counter,colons_counter,space_counter,small_letters_counter,fs_counter,digit_counter
209,26,274,64,4,4,0,184,38,4,234,56,48,62,1068,234,218,1464,11552,500,1038
225,32,74,114,2,4,14,74,10,2,46,24,12,10,530,48,40,958,6040,104,126
272,2,86,34,2,8,14,52,12,4,48,30,4,4,240,52,30,446,2974,74,218
382,10,136,82,8,14,104,52,0,0,74,22,6,32,528,80,86,850,5300,242,214
191,234,2052,554,79,126,62,400,275,6,469,544,306,38,7194,518,462,17405,82336,582,1725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,156,1338,249,80,86,15,826,132,81,607,145,162,84,2701,612,566,6818,52502,1990,2942
289,20,98,42,0,0,18,22,2,0,60,30,14,0,410,60,48,412,2784,92,108
109,336,1320,376,136,246,160,888,128,4,668,420,58,56,3660,698,654,7862,51106,1556,2952
395,243,1256,738,39,36,196,155,138,5,99,44,53,59,4678,107,203,8990,42390,367,627


##  Train Data Output

In [37]:
y_train

209    1
225    0
272    0
382    0
191    1
      ..
132    1
289    0
109    1
395    0
176    1
Name: gender, Length: 297, dtype: int64

##  Test Data Input  

In [38]:
x_test

,commas_counter,fullstop_counter,At_counter,leftbracket_counter,rightbracket_counter,exclamation_counter,minus_counter,qm_counter,per_counter,and_counter,hash_counter,US_counter,equal_counter,capital_letters_counter,sc_counter,colons_counter,space_counter,small_letters_counter,fs_counter,digit_counter
34,40,219,58,19,16,85,65,24,1,56,51,13,6,627,57,31,1986,9924,99,243
304,10,80,62,10,26,14,18,2,0,34,40,8,0,458,40,30,646,3466,50,160
151,192,500,506,36,184,96,170,98,10,576,196,56,4,3932,580,558,8140,43778,824,1260
97,318,678,539,20,25,1129,667,79,4,651,789,61,34,6422,658,404,12660,61582,824,3021
158,209,1507,1523,46,119,612,305,147,0,886,1301,120,68,9162,906,1041,12836,76084,1932,2494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,233,1423,750,19,22,224,274,139,6,479,530,137,28,5306,495,385,9108,51469,723,1858
126,310,882,148,74,82,258,218,94,14,316,874,8,4,4256,334,824,8476,44650,1050,2190
384,299,1321,21,60,64,77,220,68,3,726,48,55,8,4387,727,1063,6119,38993,1948,961
50,271,1698,198,28,29,167,703,110,8,798,427,50,24,5095,805,326,9980,58865,1159,2187


##  Test Data Output

In [39]:
y_test

34     1
304    0
151    1
97     1
158    1
      ..
387    0
126    1
384    0
50     1
24     1
Name: gender, Length: 128, dtype: int64

# Evaluate models

In [42]:
m1 = RandomForestClassifier()
m1.fit(x_train ,y_train)
p = m1.predict(x_test)
acc_rfc = accuracy_score(y_test, p)
acc_rfc

0.671875

In [45]:
m2 = LinearSVC()
m2.fit(x_train ,y_train)
p = m2.predict(x_test)
acc_lsvc = accuracy_score(y_test, p)
acc_lsvc

C:\Users\Abdullah\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.578125

# Best model

In [47]:
print('Accuracy of RandomForestClassifier:\t\t', acc_rfc)
print('Accuracy of LinearSVC:\t\t\t\t', acc_lsvc)

Accuracy of RandomForestClassifier:		 0.671875
Accuracy of LinearSVC:				 0.578125


In [48]:
print('\nBest Model is:\n')
print('Accuracy of RandomForestClassifier:\t\t\t', acc_rfc)


Best Model is:

Accuracy of RandomForestClassifier:			 0.671875


# Train Best Model

In [49]:
m1.fit(df.iloc[:,:20],df.iloc[:,20])

RandomForestClassifier()